<a href="https://colab.research.google.com/github/kksiddharth/sample/blob/master/imageEmbedding_sample123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from tensorflow.layers import flatten

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
PATH = os.getcwd()

In [3]:
PATH

'/content'

In [0]:
data_dir_list = os.listdir(PATH)
path = []
for i in data_dir_list:
    if 'jpg' in i:
        path.append(i)

In [5]:
len(path)

0

In [0]:

class Data_Set_Creator():
    
    def __init__(self):
        self.dir_list = path
        self.img_rows = 60
        self.img_cols = 60
        self.num_channel = 3
        self.num_epoch = 10
        
#        self.num_classes = 4
        self.i = 0
        
        self.img_data_list=[]
#        self.labels_list = []
        self.img_data = []
#        self.labels_name={'amazon':0,'ichiba':1,'rakuten':2,'alibaba':3}
        
    
    def set_up_data(self):
        print('Setting up Data')
        for img in self.dir_list:
            img=cv2.imread(data_path + img)
            input_img_resize=cv2.resize(img,(self.img_rows,self.img_cols))
            self.img_data_list.append(input_img_resize)
                
        self.img_data = np.array(self.img_data_list)
        self.img_data = self.img_data.astype('float32')
        self.img_data /= 255
        
        x = shuffle(self.img_data, random_state=2)
        return x

    
    def next_batch(self, x_train,batch_size):
        x = x_train[self.i:self.i+batch_size].reshape(batch_size,self.img_rows,self.img_cols,self.num_channel)
        self.i = (self.i + batch_size) % len(x_train)
        return x

    

In [7]:
obj = Data_Set_Creator()
x = obj.set_up_data()

Setting up Data


In [0]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [0]:
X_train, X_test = train_test_split(x, test_size=0.1, random_state=2)



In [12]:
initializer = tf.variance_scaling_initializer()

x = tf.placeholder(tf.float32,shape=[None,obj.img_rows,obj.img_cols,obj.num_channel])

convo_1 = convolutional_layer(x,shape=[4,4,3,32])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling,shape=[4,4,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

a = flatten(convo_2_pooling)

num_inputs = 14400
neurons_hid1 = 4096
neurons_hid2 = 1024
neurons_hid3 = 256
neurons_hid4 = neurons_hid2
neurons_hid5 = neurons_hid1 # Decoder Begins
num_outputs = num_inputs

learning_rate = 0.01

Instructions for updating:
Use keras.layers.flatten instead.


In [0]:
w1 = tf.Variable(initializer([num_inputs, neurons_hid1]), dtype=tf.float32)
w2 = tf.Variable(initializer([neurons_hid1, neurons_hid2]), dtype=tf.float32)
w3 = tf.Variable(initializer([neurons_hid2, neurons_hid3]), dtype=tf.float32)
w4 = tf.Variable(initializer([neurons_hid3, neurons_hid4]), dtype=tf.float32)
w5 = tf.Variable(initializer([neurons_hid4, neurons_hid5]), dtype=tf.float32)
w6 = tf.Variable(initializer([neurons_hid5, num_outputs]), dtype=tf.float32)

b1 = tf.Variable(tf.zeros(neurons_hid1))
b2 = tf.Variable(tf.zeros(neurons_hid2))
b3 = tf.Variable(tf.zeros(neurons_hid3))
b4 = tf.Variable(tf.zeros(neurons_hid4))
b5 = tf.Variable(tf.zeros(neurons_hid5))
b6 = tf.Variable(tf.zeros(num_outputs))

In [0]:
act_func = tf.nn.relu

hid_layer1 = act_func(tf.matmul(a, w1) + b1)
hid_layer2 = act_func(tf.matmul(hid_layer1, w2) + b2)
hid_layer3 = act_func(tf.matmul(hid_layer2, w3) + b3)
hid_layer4 = act_func(tf.matmul(hid_layer3, w4) + b4)
hid_layer5 = act_func(tf.matmul(hid_layer4, w5) + b5)
output_layer = tf.matmul(hid_layer5, w6) + b6

loss = tf.reduce_mean(tf.square(output_layer - a))


In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver() 

In [0]:
num_epochs = obj.num_epoch
#num_epochs = 10
batch_size = 150
num_samples = len(X_train)

In [0]:

with tf.Session() as sess:
    sess.run(init)
    
    # Epoch == Entire Training Set
    for epoch in range(num_epochs):
        
        num_batches = num_samples // batch_size
        
        # 150 batch size
        for iteration in range(num_batches):
            try:
                X_batch = obj.next_batch(X_train,150)
            except:
                X_batch = obj.next_batch(X_train,126)
            sess.run(train, feed_dict={x: X_batch})
            
        training_loss = loss.eval(feed_dict={x: X_batch})   
        
        print("Epoch {} Complete. Training Loss: {}".format(epoch,training_loss))
     
    saver.save(sess, "./imageEmbedding_tet123.ckpt") 

In [0]:
num_test_images = 10

with tf.Session() as sess:
    
    saver.restore(sess,"./imageEmbedding_tet123.ckpt")
    
    results = output_layer.eval(feed_dict={x:X_test[:num_test_images]})